In [1]:
import pandas as pd
import json
df = pd.read_csv(r"C:\Users\Area-51m\Desktop\sta_automation_ai\testcase-data\data\combined\new_testcase_data018.csv")

In [2]:
test_case = df[['Prompt', 'Description', 'Testcase name']].to_dict()

In [3]:
from openai import OpenAI
from textwrap import dedent

client = OpenAI(api_key='')

def generate_test_case(context):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a Robot Framework Test case generator expect, your task is to regenerate new set of User prompts, Description, and Testcase names that exist in the above examples ."},
            {"role": "user", "content": dedent(f"""
                Based on the Prompt, Description, and Testcase name, generate a new set of example that reflect the other possibilities, not just PayNow.

                Existing Test cases:
                {context}
                                
                Ensure your response is returned in a JSON FORMAT with the following keys: Prompt, Description, and Testcase name.
                Only generate one new set of example in a JSON format and don't add ```json or this ``` to the beginning or end. just give me keys and values.
            """)}
        ],
        temperature=1,
        max_tokens=100,
        top_p=1,
        frequency_penalty=0.2,
        presence_penalty=0,
        stop=["\n\n"]
    )
    
    # print(response.choices[0].message.content)
    return response.choices[0].message.content

In [ ]:
from collections import defaultdict

# Create a defaultdict with list as the default factory
my_dict = defaultdict(list)

for i in range(200):
    response = generate_test_case(test_case)
    print("RESPONSE : \n", response)
    print("\n\n\n")
    my_dict['test_cases'].append(response)
    
# Convert defaultdict to a regular dict before saving to JSON
final_dict = dict(my_dict)

In [ ]:
import json
# Save to a JSON file
with open('test_cases.json', 'w') as json_file:
    json.dump(final_dict, json_file, indent=2)

# Print the result to verify
print(json.dumps(final_dict, indent=2))

In [4]:
def generate_response(prompt, test_case):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a Robot Framework Test case generator expert, You generate accurate response for Test cases."},
            {"role": "user", "content": dedent(f"""                
                
                EXAMPLE TEXT CASES WITH COLUMN NAMES AS PROMPT, RESPONSE, Testcase name, Description:\n\n
                
                `````````````````````````````````````````````````````````````````````````````````````````````
                
                {test_case}
                
                `````````````````````````````````````````````````````````````````````````````````````````````````                

                
                `User prompt:`
                ```{prompt}```
                
                Generate a  Robot Framework Test case **Response** for User prompt, learn from the above example to unserstand what a good response looks like.
                JUST GIVE ME THE **FULL RESPONSE** FORMATTED RIGHTLY and ensure your capture all the *** Settings ***, *** Variables *** and *** Test Cases*** for the response.
                
                ONLY RESPONSE PLEASE, DON'T GIVE ME DESCRIPTION, Testcase name or add ``` signs.
                
                
            """)}
        ],
        temperature=1,
        max_tokens=3000,
        top_p=1,
        frequency_penalty=0.2,
        presence_penalty=0,
    )
    
    # print(response.choices[0].message.content)
    return response.choices[0].message.content

In [21]:
import json

# Function to read and print the content of a JSON file
def read_json_file(file_path):
    try:
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)
            # print(json.dumps(data, indent=2))  # Pretty print the JSON data
            return data
    except FileNotFoundError:
        print(f"The file {file_path} does not exist.")
    except json.JSONDecodeError:
        print(f"The file {file_path} is not a valid JSON file.")

# File path to the JSON file
file_path = 'test_cases.json'

# Read and print the JSON file content
data = read_json_file(file_path)


In [22]:
# Function to safely convert JSON strings to dictionaries
def convert_to_dict(test_cases):
    converted_cases = []
    for test_case in test_cases:
        try:
            converted_cases.append(json.loads(test_case))
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            # Optionally, append the original string or handle it as needed
    return converted_cases

# Converting each string to a dictionary
data["test_cases"] = convert_to_dict(data["test_cases"])

Error decoding JSON: Unterminated string starting at: line 4 column 20 (char 393)
Error decoding JSON: Unterminated string starting at: line 4 column 20 (char 433)
Error decoding JSON: Expecting ':' delimiter: line 2 column 162 (char 163)
Error decoding JSON: Expecting ':' delimiter: line 3 column 3 (char 161)


In [23]:
len(data["test_cases"])

266

In [35]:
from collections import defaultdict

# Create a defaultdict with list as the default factory
final_dict = defaultdict(list)

# Iterating 20 times
for idx in range(len(data['test_cases'])):
    # Sample 3 examples from dataframe
    sampled_df = df.sample(4)
    
    print(data['test_cases'][idx])
    prompt = data['test_cases'][idx]["Prompt"]
    # print(prompt)
    
    print(f"''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''{idx}'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''")
    
    # Create test_case string
    test_prompt = ""
    for _, row in sampled_df.iterrows():
        test_prompt += f"Prompt: {row['Prompt']}\nResponse: {row['Response']}\nDescription: {row['Description']}\nTestcase name: {row['Testcase name']}\n\n"
        
    # Generate response
    response = generate_response(prompt, test_prompt)
    
    # Print the results
    # print(f"Sampled Test Case:\n{test_case}")
    print(f"Generated Response:\n{response}\n")
    # print("-------------------------------------------------------------------------------------------------------------------------")
    
    # Append to the lists in final_dict
    final_dict["Prompt"].append(data['test_cases'][idx]["Prompt"])
    final_dict["Response"].append(response)
    final_dict["Description"].append(data['test_cases'][idx]["Description"])
    final_dict["Testcase name"].append(data['test_cases'][idx]["Testcase name"])


{'Prompt': 'Generate a robot framework test case for testing the ability to Validate Customer Is Able To Receive Payment Reminders functionality of the BillPro system.', 'Description': 'This test case checks the validation for the ability of a customer to receive payment reminders functionality of the BillPro system.', 'Testcase name': 'Validate Customer Receives Payment Reminders', 'Response': '*** Settings ***\nDocumentation    Test cases for validating successful login functionality using valid credentials of the BankApp system\nLibrary          SeleniumLibrary\n\n*** Variables ***\n${URL}                      https://example.com/bankapp\n${USERNAME}                 valid_username\n${PASSWORD}                 valid_password\n\n*** Test Cases ***\nValidate Successful Login With Valid Credentials\n    [Documentation]    Test case for validating successful login functionality using valid credentials of the BankApp system\n    Open Browser    ${URL}    chrome\n    Input Text    id=usern

In [37]:
test_df = pd.DataFrame(final_dict)

In [39]:
test_df.sample(10)

,Prompt,Response,Description,Testcase name
33,Generate a robot framework test case for testi...,*** Settings ***\nDocumentation Test ...,This test case validates the login functionali...,Validate Login With Expired Password On MyPay
52,Generate a robot framework test case for testi...,*** Settings ***\nDocumentation Test cases ...,This test case checks the validation for user ...,Validate Unable To Register Account With Alrea...
223,Generate a robot framework test case for testi...,*** Settings ***\nDocumentation Test cases ...,This test case checks the validation for being...,Validate Unable To Process Refunds On Transact...
31,Generate a robot framework test case for valid...,*** Settings ***\nDocumentation Test cases ...,This test case checks the validation for succe...,Validate Successful Logout Functionality
38,Generate a Robot Framework test case for testi...,*** Settings ***\nDocumentation Test ...,This test case verifies that the user account ...,Validate User Account Lockout After Multiple F...
196,Generate a robot framework test case for testi...,*** Settings ***\nDocumentation Test ...,This test case checks the validation for succe...,Validate Successful Account Recovery in SafeBank
189,Generate a robot framework test case for testi...,*** Settings ***\nDocumentation Test cases ...,This test case checks the validation of a cust...,Validate Password Reset Via Email Verification
229,Generate a robot framework test case for valid...,*** Settings ***\nDocumentation Test cases ...,This test case checks the validation process f...,Validate Customer Login With Two-Factor Authen...
172,Generate a robot framework test case for testi...,*** Settings ***\nDocumentation Test ...,This test case checks the validation for gener...,Validate Monthly Transaction Report Generation
134,Generate a robot framework test case for testi...,*** Settings ***\nDocumentation Test ...,This test case checks the validation for being...,Validate Unable To Schedule Recurring Payment ...


In [41]:
test_df.to_json("generated_data.json")